In [2]:
import numpy as np
import random
import time

# generate data

In [3]:
# In real world, you cannot learn how the data was generated. So do not rely on this function when coding your lab.
def generate_data(dim, num):
    x = np.random.normal(0, 10, [num, dim])
    coef = np.random.uniform(-1, 1, [dim, 1])
    pred = np.dot(x, coef)
    pred_n = (pred - np.mean(pred)) / np.sqrt(np.var(pred))
    label = np.sign(pred_n)
    mislabel_value = np.random.uniform(0, 1, num)
    mislabel = 0
    for i in range(num):
        if np.abs(pred_n[i]) < 1 and mislabel_value[i] > 0.9 + 0.1 * np.abs(pred_n[i]):
            label[i] *= -1
            mislabel += 1
    return x, label, mislabel/num

In [4]:
# example
x, y, mr = generate_data(5, 10000)
mr

0.0374

# write your model class

In [6]:
#in this task we will use two methods to solve the problem solve the original problem and solving the dual problem

In [14]:
# you can do anything necessary about the model
#dual
class SVM1:
    def __init__(self, dim,num):
        self.dim=dim
        self.num=num
        self.theta=np.zeros(dim)
        self.b=0
        self.a=np.zeros(num)
        self.c=10
        self.e=np.zeros(num)
    def find_j(self,best_i,X):
        valid_j_list = [i for i in range(0, len(self.a)) if self.a[i] > 0 and i != best_i]
        best_j = -1
        if len(valid_j_list) > 0:
            max_e = 0
            for j in valid_j_list:
                current_e = abs(self.e[best_i] - self.e[j])
                if current_e > max_e:
                    best_j = j
                    max_e = current_e
        else:
            # 随机选择
        # l = list(range(len(self.a)))
        # seq = l[: best_i] + l[best_i + 1:]
        # best_j = random.choice(seq)
            #选择最大的
            max_e=0
            for j in range(X.shape[0]):
                current_e=abs(self.e[j]-self.e[best_i])
                if current_e>max_e and best_i!=j:
                    best_j=j
                    max_e=current_e
        #     max_dis=0
        #     best_j=-1
        #     temp_dis=0
        #     for j in range(self.num):
        #         temp_dis=np.dot(X[j,:]-X[best_i,:],X[j,:]-X[best_i,:])
        #         if temp_dis>max_dis:
        #             best_j=j
        #             max_dis=temp_dis
        return best_j

    def meet_kkt(self,i,y_i,x_i):
        if (self.a[i]<self.c):#表明kexi=0,这个值要么
            return y_i*(np.dot(self.theta,x_i)+self.b)>=1
        else:
            return y_i*(np.dot(self.theta,x_i)+self.b)<1

    def fit(self, X, y,max_iter=50):
        iter=0
        self.c=100
        a1old=0
        a2old=0
        eta=0
        a1ujunc=0
        a2njunc=0

        # self.a=np.random.rand(self.num)
        # self.theta=np.dot(self.a,X)
        self.b=(y-np.dot(self.theta,X.T)).sum()/self.num
        self.e=np.dot(self.theta,X.T)-y+self.b
        while(iter<max_iter):
            iter=iter+1
            #print(iter)
            for i in range(self.num):
                if (self.meet_kkt(i,y[i],X[i,:])==0):
                    is_all_kkt=0
                    best_j=self.find_j(i,X)
                    x1=X[i,:]
                    x2=X[best_j,:]
                    y1=y[i]
                    y2=y[best_j]
                    e1old=self.e[i]
                    e2old=self.e[best_j]
                    eta=np.dot(x1-x2,x1-x2)
                    if (abs(eta)<1e-3):
                        continue
                    else:
                        a1old=self.a[i]
                        a2old=self.a[best_j]
                        a2njunc=a2old + y2 * (e1old - e2old)/eta#未进行修减的a2
                        if y1==y2:
                            l=max(0.,a1old+a2old-self.c)#range of a1 and a2
                            r=min(self.c,a2old+a1old)
                        else :
                            l=max(0,a2old-a1old)
                            r=min(self.c,self.c-(a2old-a1old))
                        if (a2njunc>r):
                            a2new=r
                        elif(a2njunc<l):
                            a2new=l
                        else:
                            a2new=a2njunc
                        if abs(a2new-a2old)<1e-5:
                            continue
                        else:
                            a1new=a1old+y1*y2*(a2old-a2new)
                            self.theta=self.theta+(a1new - a1old)*y1*x1+(a2new-a2old)*y2*x2
                            self.a[i]=a1new
                            self.a[best_j]=a2new
                            self.b=(y-np.dot(self.theta,X.T)).sum()/self.num
                            # b1new=y1-np.dot(self.theta, x1)
                            # b2new=y2-np.dot(self.theta, x2)
                            # if self.c > a1new > 0:
                            #     self.b=b1new
                            # elif self.c>a2new>0:
                            #     self.b=b2new
                            # else:
                            #     self.b=(b1new + b2new) / 2.0
                            self.e=np.dot(self.theta,X.T)-y+self.b
                            # for k in range(0, len(self.e)):
                            #     self.e[k] = np.dot(self.theta, X[k, :]) + self.b - y[k]
                            # self.b=-self.e.sum()/self.num
            if (is_all_kkt):
                break

            #find those to update


    def predict(self, X):
        a=X.shape[0]
        pre=np.zeros(a)
        for i in range(a):
            if (np.dot(self.theta,X[i])+self.b>0):
                pre[i]=1
            else:
                pre[i]=-1
        return pre
        """
        Use the trained model to generate prediction probabilities on a new
        collection of data points.
        """
t1=time.time()
obj1=SVM1(5,100)
obj1.fit(x,Y,max_iter=1e2)
print(obj1.theta)
t2=time.time()
# for i in range(obj.num):
#     if (obj.predict(x)[i]==Y[i]):
#         ans=ans+1
# print('acc=',ans/obj.num)
ans=0
for j in range(10):
    x,y,mr=generate_data(5,100)
    Y=np.zeros(100)
    for i in range(100):
        Y[i]=y[i]
    obj1=SVM1(5,100)
    obj1.fit(x,Y)
    for i in range(obj1.num):
        # print(np.dot(obj1.theta,x[i])+obj1.b,Y[i])
        if ((np.dot(obj1.theta,x[i])+obj1.b)*Y[i]<0):
            ans=ans+1
print('based on error picking err=',ans/obj1.num/10,'耗时',t2-t1)

[-0.11607859 -0.11009041  0.09540212 -0.04571928 -0.02659928]
based on error picking err= 0.125 耗时 0.14037489891052246


In [13]:
# you can do anything necessary about the model
class SVM2:
    def __init__(self, dim,num):
        """
        You can add some other parameters, which I think is not necessary
        """
        self.dim=dim
        self.theta=np.zeros(dim)
        self.b=0
        self.num=num
        self.episilon=0.03
    def grad(self,X,y,lr=0.0001,ep=0.05):
        J=0
        grad=np.zeros(self.dim)
        gradb=0
        for i in range(self.num):
            t=np.dot(self.theta,X[i,:])+self.b-y[i]#当y>0,t<-episilon的时候需要梯度下降当y<0,t>epsilon需要梯度下降
            if t*y[i]<-ep and y[i]>0:
                J=J+abs(t)
                grad=grad-lr*X[i,:]*y[i]
                gradb=gradb+lr*y[i]
        #grad+=self.theta*lr
        return J,grad,gradb
    def fit(self, X, y,max_iter=5e3,tol=0.003):
        """
        Fit the coefficients via your methods
        """
        J_his=[]
        J,grad,gradb=self.grad(X,y)
        iter=0
        while (J>tol and iter<max_iter):
            iter=iter+1
            J,grad,gradb=self.grad(X,y)

            self.theta=-grad+self.theta
            self.b=self.b-gradb
            if (iter>1000) and (J>J_his[-1]):
                break
            J_his.append(J)#凸优化有唯一全局解
        return J_his
    def predict(self, X):
        a=X.shape[0]
        pre=np.zeros(a)
        for i in range(a):
            if (np.dot(self.theta,X[i])+self.b>0):
                pre[i]=1
            else:
                pre[i]=-1
        return pre
        """
        Use the trained model to generate prediction probabilities on a new
        collection of data points.
        """
t1=time.time()
obj2=SVM2(5,100)
Jhis=obj2.fit(x,Y,max_iter=5e3)
#print(obj2.theta,Jhis)
ans=0
t2=time.time()
for i in range(obj2.num):
    #print(np.dot(obj1.theta,x[i])+obj1.b,Y[i])
    if ((np.dot(obj2.theta,x[i])+obj2.b)*Y[i]<=0):
        ans=ans+1
print('err=',ans/obj2.num,'\n','耗时',t2-t1)


err= 0.06 
 耗时 0.030105113983154297


# construct and train your models

In [9]:
#generate data
from sklearn import svm
def test(d,train_size):
    X_data, y_data, mislabel = generate_data(d,100+train_size)
    print('mislabel rate',mislabel)
    # split data
    X_train=X_data[:train_size,:]
    X_test=X_data[train_size:,:]
    Y_data=np.zeros(train_size+100)
    for i in range(train_size+100):
        Y_data[i]=y_data[i]
    y_train=Y_data[:train_size]
    y_test=Y_data[train_size:]
    # constrcut model and train (remember record time)
    t1=time.time()
    model2 = SVM2(d,train_size)
    model2.fit(X_train,y_train)
    t2=time.time()
    pre1=model2.predict(X_test)
    err1=0
    for i in range(y_test.size):
        if pre1[i]!=y_test[i]:
            err1=err1+1
    t_of_svr=t2-t1
    e_of_svr=err1/100
    #print('error rate of gradient descent=',err1/100,'\n','cost time',t2-t1)
    t1=time.time()
    model1 = SVM1(d,train_size)
    model1.fit(X_train,y_train)
    t2=time.time()
    pre1=model1.predict(X_test)
    err1=0
    for i in range(y_test.size):
        if pre1[i]!=y_test[i]:
            err1=err1+1
    t_of_smo=t2-t1
    e_of_smo=err1/100

    #print('error rate of SMO=',err1/100,'\n','cost time',t2-t1)

    t1=time.time()
    clf=svm.SVC(kernel="linear",C=10)
    clf.fit(X_train,y_train)
    t2=time.time()
    clf
    res=clf.predict(X_test)
    err1=0
    for i in range(X_test.shape[0]):
        if res[i]!=y_test[i]:
            err1=err1+1
    t_of_lib=t2-t1
    e_of_lib=err1/100
    e=np.zeros(3)
    t=np.zeros(3)
    e[0]=e_of_lib
    e[1]=e_of_smo
    e[2]=e_of_svr
    t[0]=t_of_lib
    t[1]=t_of_smo
    t[2]=t_of_svr
    return e,t
    #print('error rate of sklearn',err1/X_test.shape[0],'time',t2-t1)

# predict and compare your results

In [10]:
d=5
t_of_lib=np.zeros(20)
t_of_smo=np.zeros(20)
t_of_svr=np.zeros(20)
e_of_lib=np.zeros(20)
e_of_smo=np.zeros(20)
e_of_svr=np.zeros(20)
for i in range(20):
    e,t=test(d,900)
    t_of_lib[i]=t[0]
    t_of_smo[i]=t[1]
    t_of_svr[i]=t[2]
    e_of_lib[i]=e[0]
    e_of_smo[i]=e[1]
    e_of_svr[i]=e[2]
print('概况如下(重复二十次),每次维数',d,'训练集',900,'测试集100 ave of time of lib,smo,svr is',t_of_lib.mean(),t_of_smo.mean(),t_of_svr.mean(),'ave of test error rate of lib,smo,svr is',e_of_lib.mean(),e_of_smo.mean(),e_of_svr.mean())

mislabel rate 0.043
mislabel rate 0.038
mislabel rate 0.04
mislabel rate 0.035
mislabel rate 0.043
mislabel rate 0.032
mislabel rate 0.039
mislabel rate 0.041
mislabel rate 0.039
mislabel rate 0.045
mislabel rate 0.036
mislabel rate 0.034
mislabel rate 0.025
mislabel rate 0.03
mislabel rate 0.036
mislabel rate 0.042
mislabel rate 0.037
mislabel rate 0.034
mislabel rate 0.03
mislabel rate 0.041
概况如下(重复二十次),每次维数 5 训练集 9000 测试集100 ave of time of lib,smo,svr is 0.4678756594657898 0.8168471932411194 2.70404452085495 ave of test error rate of lib,smo,svr is 0.046 0.131 0.0905


In [ ]:
d=5
t_of_lib=np.zeros(20)
t_of_smo=np.zeros(20)
t_of_svr=np.zeros(20)
e_of_lib=np.zeros(20)
e_of_smo=np.zeros(20)
e_of_svr=np.zeros(20)
for i in range(20):
    e,t=test(d,3000)
    t_of_lib[i]=t[0]
    t_of_smo[i]=t[1]
    t_of_svr[i]=t[2]
    e_of_lib[i]=e[0]
    e_of_smo[i]=e[1]
    e_of_svr[i]=e[2]
print('概况如下(重复二十次),每次维数',d,'训练集',3000,'测试集100 ave of time of lib,smo,svr is',t_of_lib.mean(),t_of_smo.mean(),t_of_svr.mean(),'ave of test error rate of lib,smo,svr is',e_of_lib.mean(),e_of_smo.mean(),e_of_svr.mean())

mislabel rate 0.034516129032258064
mislabel rate 0.04548387096774194
mislabel rate 0.036774193548387096
mislabel rate 0.03741935483870968
mislabel rate 0.037096774193548385
mislabel rate 0.03354838709677419
mislabel rate 0.036774193548387096
mislabel rate 0.04387096774193548


In [12]:
d=50
t_of_lib=np.zeros(20)
t_of_smo=np.zeros(20)
t_of_svr=np.zeros(20)
e_of_lib=np.zeros(20)
e_of_smo=np.zeros(20)
e_of_svr=np.zeros(20)
for i in range(20):
    e,t=test(d,3000)
    t_of_lib[i]=t[0]
    t_of_smo[i]=t[1]
    t_of_svr[i]=t[2]
    e_of_lib[i]=e[0]
    e_of_smo[i]=e[1]
    e_of_svr[i]=e[2]
print('概况如下(重复二十次),每次维数',d,'训练集',900,'测试集100 ave of time of lib,smo,svr is',t_of_lib.mean(),t_of_smo.mean(),t_of_svr.mean(),'ave of test error rate of lib,smo,svr is',e_of_lib.mean(),e_of_smo.mean(),e_of_svr.mean())

mislabel rate 0.034193548387096775
mislabel rate 0.03870967741935484
mislabel rate 0.03903225806451613
mislabel rate 0.046129032258064515
mislabel rate 0.037096774193548385
mislabel rate 0.035483870967741936
mislabel rate 0.02903225806451613
mislabel rate 0.03193548387096774
mislabel rate 0.037096774193548385
mislabel rate 0.041935483870967745
mislabel rate 0.04225806451612903
mislabel rate 0.03645161290322581
mislabel rate 0.03870967741935484
mislabel rate 0.038387096774193545
mislabel rate 0.03870967741935484
mislabel rate 0.03387096774193549
mislabel rate 0.037096774193548385
mislabel rate 0.03129032258064516
mislabel rate 0.03645161290322581
mislabel rate 0.03483870967741935
概况如下(重复二十次),每次维数 50 训练集 900 测试集100 ave of time of lib,smo,svr is 105.18166542053223 7.095215964317322 4.616285240650177 ave of test error rate of lib,smo,svr is 0.05250000000000001 0.10900000000000003 0.10900000000000001
